In [ ]:
!pip install ultralytics

<span style="font-size:18px;">Logarithmic transformation of dark images.

In [ ]:
import os
import cv2
import numpy as np

# Input and output directories
input_directory = "./dataset/test"
output_directory = "./dataset2/test"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Get all the image filenames in the input directory
image_names = [file for file in os.listdir(input_directory) if file.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

# Process each image
for name in image_names:
    # Read the image
    image_path = os.path.join(input_directory, name)
    gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    average_pixel_value = np.mean(gray_image)
    if average_pixel_value<20:
        c = 255 / np.log(1 + np.max(gray_image))
        log_transformed_image = c * np.log(1 + gray_image)

        # Clip the pixel values to ensure they are within the valid range [0, 255]
        log_transformed_image = np.clip(log_transformed_image, 0, 255).astype(np.uint8)
        final_image = np.stack((log_transformed_image,) * 3, axis=-1)

    else:
        final_image = np.stack((gray_image,) * 3, axis=-1)
    
    # Save the transformed image to the output directory
    output_path = os.path.join(output_directory, name)
    cv2.imwrite(output_path, final_image)

print("Log transformation completed and images saved to:", output_directory)


> <span style="font-size:18px;">Converting Bbox csv annotations to yolov8 annotations

In [ ]:
#import os
import pandas as pd
import cv2 as cv

# Function to convert annotations to YOLOv5 format
def convert_to_yolov8(annotation_file, image_folder, output_folder):
    df = pd.read_csv(annotation_file)

    for index, row in df.iterrows():
        image_name = row['Name']
        class_name = row['class']
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        image_path = os.path.join(image_folder, image_name)
        image_width, image_height = get_image_size(image_path)

        x_center = (xmin + xmax) / (2 * image_width)
        y_center = (ymin + ymax) / (2 * image_height)
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        output_path = os.path.join(output_folder, image_name.replace('.jpg', '.txt'))
        with open(output_path, 'a') as out_f:
            out_f.write(f"0 {x_center} {y_center} {width} {height}\n")

# Function to get image size
def get_image_size(image_path):
    img=cv.imread(image_path)
    return img.shape[1],img.shape[0]


# Example usage
annotation_file = './dataset/bbox.csv'
image_folder = './dataset2/train'
output_folder = './dataset2/labels'

# Create output folder if not exists
os.makedirs(output_folder, exist_ok=True)

# Convert annotations
convert_to_yolov8(annotation_file, image_folder, output_folder)


<span style="font-size:18px;">Dividing Training set into training and validation set(90-10 Division)

In [ ]:
import os
import random
import shutil

# Define paths to image dataset and labels folders
image_dataset_folder = './dataset2/train'
labels_folder = './dataset2/labels'

# Define paths for training and validation folders
train_folder = './dataset3/train'
val_folder = './dataset3/val'

# Create train and val folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Create subdirectories for images and labels inside train and val folders
train_image_folder = os.path.join(train_folder, 'images')
train_label_folder = os.path.join(train_folder, 'labels')
val_image_folder = os.path.join(val_folder, 'images')
val_label_folder = os.path.join(val_folder, 'labels')

os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_image_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

# Get list of image files
image_files = os.listdir(image_dataset_folder)

# Get list of label files
label_files = os.listdir(labels_folder)

# Extract filenames without extensions
image_filenames = {os.path.splitext(filename)[0] for filename in image_files}
label_filenames = {os.path.splitext(filename)[0] for filename in label_files}

# Find common filenames between image and label datasets
common_filenames = image_filenames.intersection(label_filenames)

# Shuffle the list of common filenames to ensure randomness
common_filenames = list(common_filenames)
random.shuffle(common_filenames)

# Calculate split indices
split_index = int(0.9 * len(common_filenames))

# Split the common filenames into training and validation sets
train_filenames = common_filenames[:split_index]
val_filenames = common_filenames[split_index:]

# Copy images and labels to respective train and val folders
for filename in train_filenames:
    # Copy images
    shutil.copy(os.path.join(image_dataset_folder, filename + '.jpg'), os.path.join(train_image_folder, filename + '.jpg'))
    # Copy labels
    shutil.copy(os.path.join(labels_folder, filename + '.txt'), os.path.join(train_label_folder, filename + '.txt'))

for filename in val_filenames:
    # Copy images
    shutil.copy(os.path.join(image_dataset_folder, filename + '.jpg'), os.path.join(val_image_folder, filename + '.jpg'))
    # Copy labels
    shutil.copy(os.path.join(labels_folder, filename + '.txt'), os.path.join(val_label_folder, filename + '.txt'))

print("Dataset divided into training and validation sets successfully.")


<span style="font-size:18px;">Training process of the model

In [8]:
from ultralytics import YOLO

model = YOLO('/kaggle/working/last(2).pt')

# Train the model
# results = model.train(data='/kaggle/input/personcountvista/dataset3/data.yaml', epochs=100, patience=20,batch=-1)
results=model.train(resume=True)

Ultralytics YOLOv8.1.37 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/last(2).pt, data=/kaggle/input/personcountvista/dataset3/data.yaml, epochs=100, time=None, patience=20, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

100%|██████████| 755k/755k [00:00<00:00, 42.8MB/s]
2024-03-29 13:38:14,423	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-29 13:38:15,238	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-29 13:38:17.265435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 13:38:17.265560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 13:38:17.373980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

TensorBoard: Start with 'tensorboard --logdir runs/detect/train2', view at http://localhost:6006/


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

     48/100      11.6G      1.446     0.5848      1.291         59        640: 100%|██████████| 921/921 [17:29<00:00,  1.14s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:26<00:00,  1.94it/s]

                   all       1024       6297      0.873      0.578      0.662      0.315



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     49/100      11.5G      1.412     0.5781       1.28         92        640:   1%|▏         | 13/921 [00:15<17:52,  1.18s/it]


KeyboardInterrupt: 

Predicting test data values and storing it in CSV file

In [21]:
from ultralytics import YOLO
import pandas as pd
from IPython.display import clear_output
import os
# Load a model
model = YOLO('/kaggle/working/best.pt')  # pretrained YOLOv8n model
directory = "/kaggle/input/brightertest/test"

# Get all the filenames in the directory
image_names_relative = [file for file in os.listdir(directory)]
image_names=["/kaggle/input/wholetest/test/"+x for x in image_names_relative]
# Run batched inference on a list of images
# results = model(['./dataset/train/100.jpg', './dataset/train/14170.jpg'])  # return a list of Results objects
final=[]
for i in range(len(image_names)):
    results=model.predict(image_names[i], save=True, conf=0.35,classes=[0],stream=True)
    results=list(results)
    final.append([image_names_relative[i],len(results[0])])
    clear_output()
del image_names
# print(len(results[0]))
# print(results[0].boxes)
df = pd.DataFrame(final, columns=['Name', 'HeadCount'])

# Define the CSV file path
csv_file_path = "outputfinal.csv"

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print("CSV file has been created:", csv_file_path)

CSV file has been created: outputfinal.csv
